# Laboratorio 3 Redes neuronales profundas
#### Orlando Cabrera 19943
#### José Hurtarte 19707

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

c:\Users\josej\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pre procesamiento de datos

In [3]:
datos_mnist, info_mnist = tfds.load(name='mnist',
                                    shuffle_files = False,
                                    with_info=True, 
                                    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  50%|█████     | 2/4 [00:00<00:00,  2.28 url/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:01<00:00,  2.77 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  2.23 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:02<00:00,  1.58 url/s]


Dataset mnist downloaded and prepared to ~\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [4]:
entreno_mnist, prueba_mnist = datos_mnist['train'], datos_mnist['test']

In [5]:
num_obs_validacion = 0.1 * info_mnist.splits['train'].num_examples

In [6]:
num_obs_validacion = tf.cast(num_obs_validacion, tf.int64)

In [7]:
num_obs_prueba = info_mnist.splits['test'].num_examples

In [8]:
num_obs_prueba = tf.cast(num_obs_prueba, tf.int64)

In [9]:
def normalizar(imagen, etiqueta):
    imagen = tf.cast(imagen, tf.float32)
    imagen /= 255.
    return imagen, etiqueta

In [10]:
datos_entrenamiento_y_validacion_normalizados = entreno_mnist.map(normalizar)

In [11]:
datos_prueba = prueba_mnist.map(normalizar)

In [12]:
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_normalizados.shuffle(10000)

In [13]:
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)

In [14]:
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

In [15]:
TAMANIO_TANDA = 100

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)

In [16]:
entradas_validacion, metas_validacion = next(iter(datos_validacion))

## 1. El ancho (tamaño de la capa escondida) del algoritmo. Intenten con un tamaño de 200.  ¿Cómo cambia la precisión de validación del modelo?  ¿Cuánto  tiempo se tardó el algoritmo en entrenar?  ¿Puede encontrar un tamaño de  capa escondida que funcione mejor?

In [69]:
tamanio_capa_escondida = 200
tamanio_salida = 10

In [72]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [73]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [74]:


modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.2724 - accuracy: 0.9213 - val_loss: 0.1249 - val_accuracy: 0.9610 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1052 - accuracy: 0.9680 - val_loss: 0.0808 - val_accuracy: 0.9753 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0702 - accuracy: 0.9785 - val_loss: 0.0636 - val_accuracy: 0.9813 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0525 - accuracy: 0.9833 - val_loss: 0.0513 - val_accuracy: 0.9837 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0411 - accuracy: 0.9873 - val_loss: 0.0404 - val_accuracy: 0.9860 - 2s/epoch - 3ms/step


Presición de validación 0.9860

In [75]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 222ms/step - loss: 0.0715 - accuracy: 0.9773


In [76]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 0.07. Precisión de prueba: 97.73%


Algo bastante interesante es que a pesar de que fueran solo 2 capas escondidas y al aumentarle el numero de perceptrones por cada una de estas capas escondidas aumentó la presición de la validación de 0.9852 a 0.9860 además este siempre aumentó, por lo que a pesar del aumento de perceptrones no se observa overfit. Además hay una leve mejora en la presición de prueba del 96.69% al 97.73% por lo que podemos observar un leve aumento con respecto a las 50 neuronas por capa en el modelo inicial, además obtuvo un tiempo de ejecución de la creación del modelo de 8.6 segundos

## 2. La profundidad del algoritmo.  Agreguen una capa escondida más al algoritmo. Este es un ejercicio extremadamente importante!  ¿Cómo cambia la precisión de validación?  ¿Qué hay del tiempo que se tarda en ejecutar?   Pista:  deben tener cuidado con las formas de los pesos y los sesgos.

In [24]:
tamanio_capa_escondida = 50
tamanio_salida = 10

In [25]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3a capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [33]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [27]:
modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.4086 - accuracy: 0.8794 - val_loss: 0.1970 - val_accuracy: 0.9430 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1695 - accuracy: 0.9493 - val_loss: 0.1321 - val_accuracy: 0.9597 - 1s/epoch - 3ms/step
Epoch 3/5
540/540 - 1s - loss: 0.1293 - accuracy: 0.9610 - val_loss: 0.1167 - val_accuracy: 0.9643 - 1s/epoch - 3ms/step
Epoch 4/5
540/540 - 1s - loss: 0.1064 - accuracy: 0.9681 - val_loss: 0.0939 - val_accuracy: 0.9728 - 1s/epoch - 3ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0912 - accuracy: 0.9724 - val_loss: 0.0879 - val_accuracy: 0.9732 - 1s/epoch - 3ms/step


Presición de validación 0.9732

In [29]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 221ms/step - loss: 0.1108 - accuracy: 0.9672


In [30]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))


Pérdida de prueba: 0.11. Precisión de prueba: 96.72%


Algo que podemos mencionar de esta red es que hay un consenso que nos dice que si tenemos una red que en total tiene más de 3 capas, se suele considerar de deep learning y cuando tiene menos se considera shallow learning. Debido a esto al aumentar capas en las redes neuronales se suele mejorar la comprensión de problemas complejos. Sin embargo en este caso pudimos observar que la presición de validación es de 0.9732 lo cual no mejora la presición de validación original lo cual también no es eficiente ya que generando mayor trabajo computacional genera resultados sub optimos.

## 3. El ancho y la profundidad del algoritmo.  Agregue cuantas capas sean necesarias para llegar a 5 capas escondidas.  Es más, ajusten el ancho del algoritmo conforme lo encuentre más conveniente.  ¿Cómo cambia la precisiónde validación? ¿Qué hay del tiempo de ejecución?

Según muchos estandares, no hay una forma exacta de definir cuales son las capas y cantidad de perceptrones optimas en cada una de las capas. Hay gente que recomienda utilizar solo 1 capa oculta con la media de la entrada y la salida o gente que recomienda que el tamaño de las capas escondidas sea 2/3 de la capa de entrada mas el tamaño de la capa de salida. Sin embargo según el siguiente artículo https://www.linkedin.com/pulse/choosing-number-hidden-layers-neurons-neural-networks-sachdev/, se suele recomendar seguir el tamaño de entrada de la red neuronal por el tamaño de salida de la red neuronal y tomar la raiz cuadrada de este numero calculado cuando deseamos utilizar una red con multiples capas escondidas. Por lo que este número en nuestro caso sería:

In [32]:
int((78*78*10)**0.5)

246

Por lo que siguiendo las directrices anteriores se generó el siguiente modelo:

In [47]:
tamanio_capa_escondida = 246
tamanio_salida = 10

In [48]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 3era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 4ta capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 5ta capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [49]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [50]:
modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 3s - loss: 0.2579 - accuracy: 0.9217 - val_loss: 0.1341 - val_accuracy: 0.9597 - 3s/epoch - 6ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1047 - accuracy: 0.9676 - val_loss: 0.0783 - val_accuracy: 0.9767 - 2s/epoch - 4ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0740 - accuracy: 0.9772 - val_loss: 0.0668 - val_accuracy: 0.9800 - 2s/epoch - 4ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0595 - accuracy: 0.9818 - val_loss: 0.0722 - val_accuracy: 0.9783 - 2s/epoch - 4ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0521 - accuracy: 0.9839 - val_loss: 0.0597 - val_accuracy: 0.9830 - 2s/epoch - 4ms/step


Presición de validación 0.9830

In [51]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 260ms/step - loss: 0.0945 - accuracy: 0.9760


In [52]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))


Pérdida de prueba: 0.09. Precisión de prueba: 97.60%


Podemos observar que este modelo nos da una presición de validación del 0.9830 por lo que podemos decir que mejora con respecto a la presición de validación inicial del 0.9717 sin embargo esta se vuelve suboptima ya que al observar con las 2 capas ocultas de 200 perceptrones cada una en el inciso 1 nos da 97.73% de presición de prueba, y también la presición de validación es menor a la del inciso 1. Por eso a pesar de que tome valores bastante buenos entre los modelos probados es una solución suboptima. Sin embargo es una buena forma de generalizar caracteristicas de la red neuronal obteniendo buenos resultados

## 4. Experimenten con las funciones de activación.  Intenten aplicar una transformación sigmoidal a ambas capas.  La activación sigmoidal se obtiene escribiendo “sigmoid”.

In [77]:
tamanio_capa_escondida = 50
tamanio_salida = 10

In [78]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [80]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [81]:


modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.9933 - accuracy: 0.7845 - val_loss: 0.4197 - val_accuracy: 0.8953 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.3321 - accuracy: 0.9118 - val_loss: 0.2699 - val_accuracy: 0.9230 - 1s/epoch - 3ms/step
Epoch 3/5
540/540 - 1s - loss: 0.2436 - accuracy: 0.9312 - val_loss: 0.2156 - val_accuracy: 0.9405 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.1994 - accuracy: 0.9432 - val_loss: 0.1764 - val_accuracy: 0.9513 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.1696 - accuracy: 0.9519 - val_loss: 0.1530 - val_accuracy: 0.9577 - 1s/epoch - 2ms/step


Presición de validación 0.9577

In [82]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 238ms/step - loss: 0.1652 - accuracy: 0.9505


In [83]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 0.17. Precisión de prueba: 95.05%


Podemos decir de esta que tanto la presición de prueba (95.05%) y la de la validación (0.9577) con función de activación sigmoidal tiene resultados más bajos de estos campos que la que únicamente utiliza función de activación de unidades linealmente rectificadas en las capas escondidas. Por lo que podemos decir que utilizando función sigmoide de activación se obtienen peores resultados

## 5. Continúen experimentando con las funciones de activación.  Intenten aplicar un ReLu a la primera capa escondida y tanh a la segunda.  La activación tanh se obtiene escribiendo “tanh”.

In [84]:
tamanio_capa_escondida = 50
tamanio_salida = 10

In [85]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

In [86]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [87]:


modelo.fit(datos_entreno, 
          epochs = 5, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 0.3971 - accuracy: 0.8910 - val_loss: 0.1871 - val_accuracy: 0.9470 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1617 - accuracy: 0.9524 - val_loss: 0.1278 - val_accuracy: 0.9640 - 1s/epoch - 3ms/step
Epoch 3/5
540/540 - 1s - loss: 0.1180 - accuracy: 0.9654 - val_loss: 0.1033 - val_accuracy: 0.9700 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.0929 - accuracy: 0.9725 - val_loss: 0.0816 - val_accuracy: 0.9745 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0772 - accuracy: 0.9765 - val_loss: 0.0709 - val_accuracy: 0.9795 - 1s/epoch - 2ms/step


In [88]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 205ms/step - loss: 0.0908 - accuracy: 0.9718


In [89]:
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 0.09. Precisión de prueba: 97.18%


Como se puede observar, se obtuvieron resultados ligeramente mejores que en la corrida original. Así que cambiando la función de activación tanh da mejores resultados en general que usando la función sigmoidal o la función lineal rectificada (relu). 

## 6. Ajusten el tamaño de la tanda.  Prueben con un tamaño de tanda de 10,000. ¿Cómo cambia el tiempo requerido?  ¿Cómo cambia la precisión? 

In [ ]:
datos_entrenamiento_y_validacion_normalizados = entreno_mnist.map(normalizar)
datos_prueba = prueba_mnist.map(normalizar)
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_normalizados.shuffle(TAMANIO_BUFFER)
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

In [ ]:
TAMANIO_TANDA = 10000

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)
entradas_validacion, metas_validacion = next(iter(datos_validacion))

In [ ]:
tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno, 
          epochs = NUMERO_EPOCAS, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
6/6 - 3s - loss: 0.6635 - accuracy: 0.8305 - val_loss: 0.5902 - val_accuracy: 0.8432 - 3s/epoch - 493ms/step
Epoch 2/5
6/6 - 3s - loss: 0.5564 - accuracy: 0.8512 - val_loss: 0.5033 - val_accuracy: 0.8623 - 3s/epoch - 484ms/step
Epoch 3/5
6/6 - 3s - loss: 0.4814 - accuracy: 0.8676 - val_loss: 0.4414 - val_accuracy: 0.8742 - 3s/epoch - 487ms/step
Epoch 4/5
6/6 - 3s - loss: 0.4269 - accuracy: 0.8817 - val_loss: 0.3968 - val_accuracy: 0.8878 - 3s/epoch - 489ms/step
Epoch 5/5
6/6 - 3s - loss: 0.3895 - accuracy: 0.8915 - val_loss: 0.3635 - val_accuracy: 0.8963 - 3s/epoch - 482ms/step
1/1 [==============================] - 0s 327ms/step - loss: 0.3594 - accuracy: 0.8993
Pérdida de prueba: 0.36. Precisión de prueba: 89.93%


Como se puede observar, el cambiar el tamaño de la tanda sí modificó el resultado, teniendo una presición de 89.93% y una perdida de 0.36, siendo estos valores bajos respectos a los resultados originales, en donde se utilizó un tamaño de la tanda de 100 por default. Pero gracias a que se utilizó un tamaño mayor de tanda en este caso, se logró reducir el tiempo de ejecución aproximadamente 10 veces menos que en la corrida original. Pasando de tardarse 4 segundos por cada época, a aproximadamente 500 milisegundos en cada epoca al aumentar el tamaño de la tanda a 10000

## 7. Ajusten el tamaño de la tanda a 1.  Eso corresponde al SGD. ¿Cómo cambian el tiempo y la precisión?  ¿Es el resultado coherente con la teoría?

In [ ]:
datos_entrenamiento_y_validacion_normalizados = entreno_mnist.map(normalizar)
datos_prueba = prueba_mnist.map(normalizar)
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_normalizados.shuffle(TAMANIO_BUFFER)
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

In [ ]:
TAMANIO_TANDA = 1

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)
entradas_validacion, metas_validacion = next(iter(datos_validacion))

In [ ]:
tamanio_capa_escondida = 50
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno, 
          epochs = NUMERO_EPOCAS, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
54000/54000 - 72s - loss: 0.2434 - accuracy: 0.9272 - val_loss: 0.1474 - val_accuracy: 0.9572 - 72s/epoch - 1ms/step
Epoch 2/5
54000/54000 - 80s - loss: 0.1504 - accuracy: 0.9582 - val_loss: 0.1493 - val_accuracy: 0.9628 - 80s/epoch - 1ms/step
Epoch 3/5
54000/54000 - 80s - loss: 0.1353 - accuracy: 0.9636 - val_loss: 0.1402 - val_accuracy: 0.9670 - 80s/epoch - 1ms/step
Epoch 4/5
54000/54000 - 83s - loss: 0.1297 - accuracy: 0.9662 - val_loss: 0.1325 - val_accuracy: 0.9638 - 83s/epoch - 2ms/step
Epoch 5/5
54000/54000 - 83s - loss: 0.1240 - accuracy: 0.9690 - val_loss: 0.1403 - val_accuracy: 0.9675 - 83s/epoch - 2ms/step
1/1 [==============================] - 0s 480ms/step - loss: 0.1671 - accuracy: 0.9621
Pérdida de prueba: 0.17. Precisión de prueba: 96.21%


Para este caso, el tiempo aumentó considerablemente, aproximadamente 20 veces más que en la corrida original. Además, la precisión y la perdida que se obtuvo en este caso no tuvo un cambio significativo respecto a la corrida original, siendo esta corrida ligeramente peor que la original, pero siendo buenos resultados en general. Gracias a esta pregunta y la anterior, se puede concluir que el aumentar o disminuir el tamaño de las tandas en exceso no es sinónimo de obtener mejores resultados. En algunos caso sí podría dar mejores resultados, pero eso significaría utilizar más recursos computacionales.

## 8. Ajusten la tasa de aprendizaje.  Prueben con un valor de 0.0001.  ¿Hace alguna diferencia?

In [ ]:
datos_entrenamiento_y_validacion_normalizados = entreno_mnist.map(normalizar)
datos_prueba = prueba_mnist.map(normalizar)
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_normalizados.shuffle(TAMANIO_BUFFER)
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

In [ ]:
TAMANIO_TANDA = 100

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)
entradas_validacion, metas_validacion = next(iter(datos_validacion))

In [ ]:
tamanio_capa_escondida = 50
tasa_aprendizaje = 0.0001
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
optimizador_adhoc = tf.keras.optimizers.SGD(learning_rate=tasa_aprendizaje)
modelo.compile(optimizer = optimizador_adhoc, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno, 
          epochs = NUMERO_EPOCAS, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
540/540 - 5s - loss: 2.3346 - accuracy: 0.0996 - val_loss: 2.3246 - val_accuracy: 0.0990 - 5s/epoch - 8ms/step
Epoch 2/5
540/540 - 4s - loss: 2.3110 - accuracy: 0.1034 - val_loss: 2.3029 - val_accuracy: 0.1030 - 4s/epoch - 8ms/step
Epoch 3/5
540/540 - 6s - loss: 2.2911 - accuracy: 0.1097 - val_loss: 2.2836 - val_accuracy: 0.1097 - 6s/epoch - 11ms/step
Epoch 4/5
540/540 - 7s - loss: 2.2727 - accuracy: 0.1174 - val_loss: 2.2656 - val_accuracy: 0.1190 - 7s/epoch - 12ms/step
Epoch 5/5
540/540 - 5s - loss: 2.2554 - accuracy: 0.1279 - val_loss: 2.2486 - val_accuracy: 0.1315 - 5s/epoch - 10ms/step
1/1 [==============================] - 1s 518ms/step - loss: 2.2447 - accuracy: 0.1386
Pérdida de prueba: 2.24. Precisión de prueba: 13.86%


El cambiar la tasa de aprendizaje a 0.0001 sí realizó cambios muy grandes en el resultados, siendo que se obtuvo una precisión y una perdida demasiados malos. Así que se puede decir que el disminuir la tasa de aprendizaje, para este caso, da resultados muy malos.

## 9. Ajusten la tasa de aprendizaje a 0.02.  ¿Hay alguna diferencia?

In [ ]:
datos_entrenamiento_y_validacion_normalizados = entreno_mnist.map(normalizar)
datos_prueba = prueba_mnist.map(normalizar)
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_normalizados.shuffle(TAMANIO_BUFFER)
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

In [ ]:
TAMANIO_TANDA = 100

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)
entradas_validacion, metas_validacion = next(iter(datos_validacion))

In [ ]:
tamanio_capa_escondida = 50
tasa_aprendizaje = 0.02
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
optimizador_adhoc = tf.keras.optimizers.SGD(learning_rate=tasa_aprendizaje)
modelo.compile(optimizer = optimizador_adhoc, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno, 
          epochs = NUMERO_EPOCAS, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
540/540 - 6s - loss: 0.9528 - accuracy: 0.7411 - val_loss: 0.4663 - val_accuracy: 0.8760 - 6s/epoch - 11ms/step
Epoch 2/5
540/540 - 6s - loss: 0.3981 - accuracy: 0.8899 - val_loss: 0.3562 - val_accuracy: 0.9013 - 6s/epoch - 10ms/step
Epoch 3/5
540/540 - 5s - loss: 0.3337 - accuracy: 0.9054 - val_loss: 0.3190 - val_accuracy: 0.9102 - 5s/epoch - 10ms/step
Epoch 4/5
540/540 - 5s - loss: 0.3029 - accuracy: 0.9133 - val_loss: 0.2927 - val_accuracy: 0.9192 - 5s/epoch - 10ms/step
Epoch 5/5
540/540 - 5s - loss: 0.2761 - accuracy: 0.9203 - val_loss: 0.2774 - val_accuracy: 0.9235 - 5s/epoch - 10ms/step
1/1 [==============================] - 1s 693ms/step - loss: 0.2625 - accuracy: 0.9271
Pérdida de prueba: 0.26. Precisión de prueba: 92.71%


En este caso, al aumentar la tasa de aprendizaje al doble (de 0.01 a 0.02) se obtuvieron resultados más bajos, pero si se comparan con los resultados de la pregunta anterior, se obtuvieron resultados mucho mejores. Así que para este conjunto de datos, es mejor utilizar tasas de aprendizaje aproximados al 0.01.

## 10. Combinen todos los métodos indicados arriba e intenten llegar a una precisión de validación de 98.5% o más. 

In [ ]:
datos_entrenamiento_y_validacion_normalizados = entreno_mnist.map(normalizar)
datos_prueba = prueba_mnist.map(normalizar)
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_normalizados.shuffle(TAMANIO_BUFFER)
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

In [ ]:
TAMANIO_TANDA = 100

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)
entradas_validacion, metas_validacion = next(iter(datos_validacion))

In [ ]:
tamanio_capa_escondida = 200
tasa_aprendizaje = 0.01
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='relu'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='tanh'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])
optimizador_adhoc = tf.keras.optimizers.SGD(learning_rate=tasa_aprendizaje)
modelo.compile(optimizer = "adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno, 
          epochs = NUMERO_EPOCAS, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Epoch 1/5
540/540 - 8s - loss: 0.2567 - accuracy: 0.9245 - val_loss: 0.1405 - val_accuracy: 0.9582 - 8s/epoch - 15ms/step
Epoch 2/5
540/540 - 8s - loss: 0.0998 - accuracy: 0.9695 - val_loss: 0.0850 - val_accuracy: 0.9752 - 8s/epoch - 14ms/step
Epoch 3/5
540/540 - 8s - loss: 0.0656 - accuracy: 0.9790 - val_loss: 0.0700 - val_accuracy: 0.9800 - 8s/epoch - 14ms/step
Epoch 4/5
540/540 - 7s - loss: 0.0462 - accuracy: 0.9853 - val_loss: 0.0583 - val_accuracy: 0.9832 - 7s/epoch - 14ms/step
Epoch 5/5
540/540 - 7s - loss: 0.0365 - accuracy: 0.9884 - val_loss: 0.0456 - val_accuracy: 0.9870 - 7s/epoch - 14ms/step
1/1 [==============================] - 1s 695ms/step - loss: 0.0752 - accuracy: 0.9756
Pérdida de prueba: 0.08. Precisión de prueba: 97.56%


Como se puede observar, se logró tener una precisión de validación de 98.70%. Esto se consiguió cambiando el tamaño de la capa escondida a 200, cambiando la función de activación de la segunda capa a tanh y dejando los demás parámetros iguales. No se decidió cambiar ningún otro parámetro debido a que, como se pudo observar en las preguntas anteriores, siempre se obtenía un valor más bajo que en la corrida original, siendo únicamente el cambio del tamaño de la capa esconidad y el da la función de activación los que dieron un resultado mejor. 

Además, muchos libros de texto suelen recomendar numeros de perceptrones que estén entre valores de la capa de salida y entrada, debido a esto se escogió un valor de 200 para el tamaño de la capa escondida, lo cual fue algo que funcionó muy bien cuando se utilizó con función de activación lineal rectificada (relu)